---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [82]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [83]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.index[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

'Afghanistan'

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [84]:
def answer_one():
    return df[df['Gold']==max(df['Gold'])].index.get_values()[0]

In [85]:
answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [86]:
def answer_two():
    df['diff'] = abs(df['Gold'] - df['Gold.1'])
    return df['diff'].idxmax()

In [87]:
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [88]:
def answer_three():
    df_1 = df[(df['Gold'] > 0) & (df['Gold.1'] > 0) ]
    df_1['diff_1'] = (df_1['Gold'] - df_1['Gold.1'])/df_1['Gold.2']
    return df_1['diff_1'].idxmax()

In [89]:
answer_three()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [90]:
def answer_four():
    df['Points'] = df['Gold.2'] * 3.0 + df['Silver.2'] * 2.0 + df['Bronze.2'] * 1.0
    return df['Points']

In [91]:
answer_four()

Afghanistan                            2.0
Algeria                               27.0
Argentina                            130.0
Armenia                               16.0
Australasia                           22.0
Australia                            923.0
Austria                              569.0
Azerbaijan                            43.0
Bahamas                               24.0
Bahrain                                1.0
Barbados                               1.0
Belarus                              154.0
Belgium                              276.0
Bermuda                                1.0
Bohemia                                5.0
Botswana                               2.0
Brazil                               184.0
British West Indies                    2.0
Bulgaria                             411.0
Burundi                                3.0
Cameroon                              12.0
Canada                               846.0
Chile                                 24.0
China      

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [92]:
census_df = pd.read_csv('census.csv')
census_df.head(100)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411
6,50,3,6,1,11,Alabama,Bullock County,10914,10915,10887,...,-30.953709,-5.180127,-1.130263,14.354290,-16.167247,-29.001673,-2.825524,1.507017,17.243790,-13.193961
7,50,3,6,1,13,Alabama,Butler County,20947,20946,20944,...,-14.032727,-11.684234,-5.655413,1.085428,-6.529805,-13.936612,-11.586865,-5.557058,1.184103,-6.430868
8,50,3,6,1,15,Alabama,Calhoun County,118572,118586,118437,...,-6.155670,-4.611706,-5.524649,-4.463211,-3.376322,-5.791579,-4.092677,-5.062836,-3.912834,-2.806406
9,50,3,6,1,17,Alabama,Chambers County,34215,34170,34098,...,-2.731639,3.849092,2.872721,-2.287222,1.349468,-1.821092,4.701181,3.781439,-1.290228,2.346901


In [93]:
def answer_five():
    return census_df[['STNAME','COUNTY']].groupby(['STNAME']).sum()['COUNTY'].idxmax()

In [94]:
answer_five()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [ ]:
# def answer_six():
#     counties_df = census_df[census_df['SUMLEV'] == 50]
#     top_counties_df = counties_df.sort_values(by=['STNAME','CENSUS2010POP'],ascending=False).groupby('STNAME').head(3)
#     ans = top_counties_df.groupby('STNAME').sum().sort_values(by='CENSUS2010POP').head(3).index.tolist()
#     return ans
# answer_six()

In [126]:
#counties_df = census_df[census_df['SUMLEV'] == 50]

In [127]:
# top_counties_df = counties_df.sort_values(by=['STNAME','CENSUS2010POP'],ascending=False).groupby('STNAME').head(3)
# top_counties_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
3180,50,4,8,56,21,Wyoming,Laramie County,91738,91881,92271,...,-1.200428,15.547274,4.787847,-1.226133,0.278940,-0.973320,17.914554,6.003143,-0.207819,1.673640
3182,50,4,8,56,25,Wyoming,Natrona County,75450,75450,75472,...,7.189319,23.066162,24.322042,-0.958472,-0.061057,7.689674,23.749508,25.085233,-0.110593,0.793743
3172,50,4,8,56,5,Wyoming,Campbell County,46133,46133,46244,...,-2.843479,15.601020,-5.895711,-8.550911,10.916963,-2.649606,15.558684,-5.916543,-8.509402,10.978525
3137,50,2,3,55,79,Wisconsin,Milwaukee County,947735,947736,948301,...,-4.417899,-5.257085,-5.581617,-8.303054,-8.604921,-2.352634,-2.953702,-3.114494,-5.344614,-5.623163
3109,50,2,3,55,25,Wisconsin,Dane County,488073,488075,489187,...,5.468267,4.529574,3.392786,3.107322,2.481328,8.430668,7.729273,6.874337,7.038202,6.383028


In [145]:
def answer_six():
    census_df_0 = census_df[census_df['SUMLEV'] == 50]
    census_df_1 = census_df_0.groupby(['STNAME'])['CENSUS2010POP'].nlargest(3).reset_index()
    census_df_2 = census_df_1.groupby('STNAME').sum().sort_values('CENSUS2010POP', ascending = False)['CENSUS2010POP'].head(3)
    return list(census_df_2.index)

In [146]:
answer_six()

['California', 'Texas', 'Illinois']

In [135]:
census_df_0 = census_df[census_df['SUMLEV'] == 50]

In [138]:
census_df_1 = census_df_0.groupby(['STNAME'])['CENSUS2010POP'].nlargest(3)
census_df_1.head()
#census_df_1.groupby('STNAME').sum()

STNAME     
Alabama  37    658466
         49    412992
         45    334811
Alaska   71    291826
         76     97581
Name: CENSUS2010POP, dtype: int64

In [134]:
census_df[census_df['STNAME'] == 'Wyoming']

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
3169,40,4,8,56,0,Wyoming,Wyoming,563626,563767,564516,...,-0.381530,9.636214,4.487115,-4.788275,-3.221091,0.289680,10.694870,5.440390,-3.727831,-2.091573
3170,50,4,8,56,1,Wyoming,Albany County,36299,36299,36428,...,3.708956,2.637812,-3.544634,-3.334877,-9.911169,6.736119,6.433032,0.719587,1.429233,-5.166460
3171,50,4,8,56,3,Wyoming,Big Horn County,11668,11668,11672,...,4.868258,2.804930,16.815908,-8.026420,5.095861,4.868258,3.144921,17.236306,-7.608378,5.513554
3172,50,4,8,56,5,Wyoming,Campbell County,46133,46133,46244,...,-2.843479,15.601020,-5.895711,-8.550911,10.916963,-2.649606,15.558684,-5.916543,-8.509402,10.978525
3173,50,4,8,56,7,Wyoming,Carbon County,15885,15885,15837,...,-7.581980,-13.081441,3.178134,-2.970641,-23.300971,-7.392431,-12.636926,3.623073,-2.338590,-22.600668
3174,50,4,8,56,9,Wyoming,Converse County,13833,13833,13826,...,-12.847499,15.493820,19.035533,-20.550587,-0.070403,-12.774915,16.502720,20.093063,-19.358233,1.126443
3175,50,4,8,56,11,Wyoming,Crook County,7083,7083,7114,...,-1.544618,-4.202564,1.397819,6.378258,18.629317,-0.982939,-3.642222,2.096729,7.071547,19.309219
3176,50,4,8,56,13,Wyoming,Fremont County,40123,40123,40222,...,2.747083,7.782673,-4.990688,-12.331633,-13.673610,3.093562,8.027411,-4.747240,-12.013555,-13.352750
3177,50,4,8,56,15,Wyoming,Goshen County,13249,13247,13408,...,14.293649,3.961413,-8.079028,-7.017803,-11.899450,14.886132,4.841727,-6.903896,-5.761986,-10.635133
3178,50,4,8,56,17,Wyoming,Hot Springs County,4812,4812,4813,...,3.322604,6.208609,3.095336,-6.017222,-5.454164,5.191569,6.001656,2.888981,-6.224712,-5.663940


In [139]:
census_df_1 = census_df_1.reset_index()
census_df_1.head()

,STNAME,level_1,CENSUS2010POP
0,Alabama,37,658466
1,Alabama,49,412992
2,Alabama,45,334811
3,Alaska,71,291826
4,Alaska,76,97581


In [144]:
census_df_1[census_df_1['STNAME'] == 'Wyoming']

,STNAME,level_1,CENSUS2010POP
148,Wyoming,3180,91738
149,Wyoming,3182,75450
150,Wyoming,3172,46133


### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [159]:
def answer_seven():
    census_df_0 = census_df[census_df['SUMLEV'] == 50]
    census_df_1 = census_df_0[['CTYNAME', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']]
    census_df_2 = census_df_1.set_index('CTYNAME')
    diff_df = census_df_2.max(axis = 1) - census_df_2.min(axis = 1)
    return diff_df.sort_values(ascending = False).index[0]

In [149]:
census_df_0 = census_df[census_df['SUMLEV'] == 50]

In [151]:
census_df_1 = census_df_0[['CTYNAME', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']]
census_df_1.head()

,CTYNAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
1,Autauga County,54660,55253,55175,55038,55290,55347
2,Baldwin County,183193,186659,190396,195126,199713,203709
3,Barbour County,27341,27226,27159,26973,26815,26489
4,Bibb County,22861,22733,22642,22512,22549,22583
5,Blount County,57373,57711,57776,57734,57658,57673


In [153]:
census_df_2 = census_df_1.set_index('CTYNAME')
census_df_2.head()

,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
CTYNAME,,,,,,
Autauga County,54660,55253,55175,55038,55290,55347
Baldwin County,183193,186659,190396,195126,199713,203709
Barbour County,27341,27226,27159,26973,26815,26489
Bibb County,22861,22733,22642,22512,22549,22583
Blount County,57373,57711,57776,57734,57658,57673


In [156]:
diff_df = census_df_2.max(axis = 1) - census_df_2.min(axis = 1)
diff_df.head()

CTYNAME
Autauga County      687
Baldwin County    20516
Barbour County      852
Bibb County         349
Blount County       403
dtype: int64

In [158]:
diff_df.sort_values(ascending = False).index[0]

'Harris County'

In [160]:
answer_seven()

'Harris County'

In [103]:
census_df_2 = census_df[['CTYNAME', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']]

In [104]:
census_df_2.head()

,CTYNAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
0,Alabama,4785161,4801108,4816089,4830533,4846411,4858979
1,Autauga County,54660,55253,55175,55038,55290,55347
2,Baldwin County,183193,186659,190396,195126,199713,203709
3,Barbour County,27341,27226,27159,26973,26815,26489
4,Bibb County,22861,22733,22642,22512,22549,22583


In [105]:
census_df_3 = census_df_2.set_index('CTYNAME')
census_df_3.head(10)

,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
CTYNAME,,,,,,
Alabama,4785161,4801108,4816089,4830533,4846411,4858979
Autauga County,54660,55253,55175,55038,55290,55347
Baldwin County,183193,186659,190396,195126,199713,203709
Barbour County,27341,27226,27159,26973,26815,26489
Bibb County,22861,22733,22642,22512,22549,22583
Blount County,57373,57711,57776,57734,57658,57673
Bullock County,10887,10629,10606,10628,10829,10696
Butler County,20944,20673,20408,20261,20276,20154
Calhoun County,118437,117768,117286,116575,115993,115620


In [106]:
a = census_df_3.max(axis = 1).head(10)
a

CTYNAME
Alabama            4858979
Autauga County       55347
Baldwin County      203709
Barbour County       27341
Bibb County          22861
Blount County        57776
Bullock County       10887
Butler County        20944
Calhoun County      118437
Chambers County      34153
dtype: int64

In [107]:
b = census_df_3.min(axis = 1).head(10)
b

CTYNAME
Alabama            4785161
Autauga County       54660
Baldwin County      183193
Barbour County       26489
Bibb County          22512
Blount County        57373
Bullock County       10606
Butler County        20154
Calhoun County      115620
Chambers County      33993
dtype: int64

In [108]:
c = a - b
c.sort(ascending = False)
c.index[0]

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
  from ipykernel import kernelapp as app


'Alabama'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [109]:
census_df_3 = census_df[census_df['REGION'].isin([1, 2]) & (census_df['CTYNAME'].str.contains('Washington')) & (census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014'])]

In [110]:
census_df_3[['STNAME', 'CTYNAME']]

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County


In [111]:
census_df.head(100)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411
6,50,3,6,1,11,Alabama,Bullock County,10914,10915,10887,...,-30.953709,-5.180127,-1.130263,14.354290,-16.167247,-29.001673,-2.825524,1.507017,17.243790,-13.193961
7,50,3,6,1,13,Alabama,Butler County,20947,20946,20944,...,-14.032727,-11.684234,-5.655413,1.085428,-6.529805,-13.936612,-11.586865,-5.557058,1.184103,-6.430868
8,50,3,6,1,15,Alabama,Calhoun County,118572,118586,118437,...,-6.155670,-4.611706,-5.524649,-4.463211,-3.376322,-5.791579,-4.092677,-5.062836,-3.912834,-2.806406
9,50,3,6,1,17,Alabama,Chambers County,34215,34170,34098,...,-2.731639,3.849092,2.872721,-2.287222,1.349468,-1.821092,4.701181,3.781439,-1.290228,2.346901


In [114]:
def answer_eight():
    census_df_3 = census_df[census_df['REGION'].isin([1, 2]) & (census_df['CTYNAME'].str.contains('Washington')) & (census_df['POPESTIMATE2015'] > census_df['POPESTIMATE2014'])]
    return census_df_3.loc[:,['STNAME', 'CTYNAME']]

In [117]:
answer_eight().shape

(5, 2)